In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [2]:
DATA_PATH = os.path.join('dataset')
actions = np.array(['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine'])
sequence_length = 100
num_features = 126
label_map = {label:num for num, label in enumerate(actions)}

In [3]:
sequences, labels = [], []

for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    for sequence in np.array(os.listdir(action_path)).astype(int):
        frame_paths = [
            os.path.join(action_path, str(sequence), "{}.npy".format(frame_num))
            for frame_num in range(sequence_length)
        ]
        window = [np.load(frame_path) for frame_path in frame_paths]
        sequences.append(window)
        labels.append(label_map[action])

In [4]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30)

In [5]:
log_dir = os.path.join('VGG16_Logs')
callback = TensorBoard(log_dir=log_dir)
ACCURACY_THRESHOLD = 0.95

class MyCallback(tf.keras.callbacks.Callback): 
    def __init__(self, monitor_metric='accuracy'):
        super(MyCallback, self).__init__()
        self.monitor_metric = monitor_metric
        self.threshold = ACCURACY_THRESHOLD

    def on_epoch_end(self, epoch, logs={}): 
        current_metric_value = logs.get(self.monitor_metric)
        if current_metric_value is not None and current_metric_value > self.threshold:
            print(f"\nReached {self.threshold * 100:.2f}% {self.monitor_metric}, stopping training!")
            self.model.stop_training = True

tb_callback = TensorBoard(log_dir=log_dir)
my_callback = MyCallback(monitor_metric='accuracy')

In [6]:
# Define the model
model = Sequential()

# Block 1
model.add(BatchNormalization())
model.add(Conv1D(64, 3, activation='relu', padding='same', input_shape=(sequence_length, num_features)))
model.add(MaxPooling1D(2, strides=2))

# Block 2
model.add(BatchNormalization())
model.add(Conv1D(128, 3, activation='relu', padding='same'))
model.add(MaxPooling1D(2, strides=2))

# Block 3
model.add(BatchNormalization())
model.add(Conv1D(256, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(256, 3, activation='relu', padding='same'))
model.add(MaxPooling1D(2, strides=2))

# Block 4
model.add(BatchNormalization())
model.add(Conv1D(512, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(512, 3, activation='relu', padding='same'))
model.add(MaxPooling1D(2, strides=2))

# Block 5
model.add(BatchNormalization())
model.add(Conv1D(512, 3, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv1D(512, 3, activation='relu', padding='same'))
model.add(MaxPooling1D(2, strides=2))

# Flatten
model.add(Flatten())

# Fully Connected Layers
model.add(BatchNormalization())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

model.add(BatchNormalization())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(actions.shape[0], activation='softmax'))

In [7]:
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name="Adam")
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [9]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=16, callbacks=[tb_callback, my_callback, early_stopping])

Epoch 1/100
88/88 [==============================] - 9s 31ms/step - loss: 2.4018 - accuracy: 0.2407 - val_loss: 2.4496 - val_accuracy: 0.1017
Epoch 2/100
88/88 [==============================] - 2s 22ms/step - loss: 1.7049 - accuracy: 0.4493 - val_loss: 2.8303 - val_accuracy: 0.1050
Epoch 3/100
88/88 [==============================] - 2s 21ms/step - loss: 1.6033 - accuracy: 0.4779 - val_loss: 2.6186 - val_accuracy: 0.2183
Epoch 4/100
88/88 [==============================] - 2s 22ms/step - loss: 1.4403 - accuracy: 0.5371 - val_loss: 2.3800 - val_accuracy: 0.2900
Epoch 5/100
88/88 [==============================] - 2s 22ms/step - loss: 1.2549 - accuracy: 0.5979 - val_loss: 1.9148 - val_accuracy: 0.4100
Epoch 6/100
88/88 [==============================] - 2s 22ms/step - loss: 1.1603 - accuracy: 0.6271 - val_loss: 1.7599 - val_accuracy: 0.4917
Epoch 7/100
88/88 [==============================] - 2s 22ms/step - loss: 1.1895 - accuracy: 0.6264 - val_loss: 1.0149 - val_accuracy: 0.6567
Epoch 

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 100, 126)         504       
 ormalization)                                                   
                                                                 
 conv1d (Conv1D)             (None, 100, 64)           24256     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 50, 64)           0         
 )                                                               
                                                                 
 batch_normalization_1 (Batc  (None, 50, 64)           256       
 hNormalization)                                                 
                                                                 
 conv1d_1 (Conv1D)           (None, 50, 128)           24704     
                                                        

In [15]:
model.save('VGG16.h5')

In [20]:
model.load_weights('VGG16.h5')

In [11]:
yhat = model.predict(x_train)
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

print("Train Accuracy :-> ")
print(accuracy_score(ytrue, yhat)*100)

44/44 [==============================] - 1s 6ms/step
Train Accuracy :-> 
88.18256475674653


In [12]:
yhat = model.predict(x_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

print("Test Accuracy :-> ")
print(accuracy_score(ytrue, yhat)*100)

19/19 [==============================] - 0s 14ms/step
Test Accuracy :-> 
83.33333333333333
